###  Created by Luis A. Sanchez-Perez (l.alejandro.2011@gmail.com).
<p><span style="color:green"><b>Copyright &#169;</b> Do not distribute or use without authorization from author.</span></p>

In [1]:
import tensorflow as tf
import pandas as pd
import pathlib
import os
import numpy as np

In [2]:
DATASETS = pathlib.Path(os.environ['DATASETS'])

In [3]:
model = tf.saved_model.load('models/NeuralMF/20210402-110456/1/')

In [4]:
user_id = 25

In [5]:
ratings = pd.read_csv(DATASETS / 'recommender/movies/ml-100k/u.data',
                      sep='\t', names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')

In [6]:
ratings.set_index(keys=['userId', 'movieId'], inplace=True)

In [7]:
movies = set(ratings.index.get_level_values(level=1))

In [8]:
rated = set(ratings.loc[user_id].index)

In [9]:
unrated = list(movies - rated)

In [10]:
ratings = model.get_movies_ranking_for_user(
    tf.constant(user_id, dtype=tf.int32),
    tf.convert_to_tensor(unrated, dtype=tf.int32)
)

In [11]:
rank = [movie for _, movie in sorted(zip(ratings['scores'].numpy().flatten(), unrated), reverse=True)]

In [12]:
genre_columns = np.array([
    "Unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
])
movies_columns = [
    'movieId', 'title', 'release_date', 'video_release_date', 'imdb_url'
] + genre_columns.tolist()
    
def load_movies_data():
    movies = pd.read_csv(
        DATASETS / 'recommender/movies/ml-100k/u.item', 
        sep='|', names=movies_columns, 
        index_col=['movieId'],
        encoding='latin-1'
    )
    movies['genre'] = ['|'.join(genre_columns[row]) for row in movies[genre_columns].values.astype(bool)]
    movies.drop(columns=['release_date', 'video_release_date', 'imdb_url'] + genre_columns.tolist(), inplace=True)
    return movies

In [13]:
movies = load_movies_data()
movies.head()

,title,genre
movieId,,
1,Toy Story (1995),Animation|Children|Comedy
2,GoldenEye (1995),Action|Adventure|Thriller
3,Four Rooms (1995),Thriller
4,Get Shorty (1995),Action|Comedy|Drama
5,Copycat (1995),Crime|Drama|Thriller


In [14]:
movies.loc[rank]

,title,genre
movieId,,
778,Don Juan DeMarco (1995),Comedy|Drama|Romance
756,Father of the Bride Part II (1995),Comedy
747,Benny & Joon (1993),Comedy|Romance
732,Dave (1993),Comedy|Romance
709,Strictly Ballroom (1992),Comedy|Romance
...,...,...
329,Desperate Measures (1998),Crime|Drama|Thriller
1025,Fire Down Below (1997),Action|Drama|Thriller
236,Citizen Ruth (1996),Comedy|Drama
